In [28]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import shapely
from fiona.crs import from_epsg
from shapely.ops import cascaded_union
from shapely.geometry import Point, LineString
from shapely.geometry import mapping
from shapely.ops import nearest_points
from tqdm import tqdm, trange

In [29]:
def nearest(row, geom_union, df1, df2, geom1_col='geometry', geom2_col='geometry', src_column=None):
    """Find the nearest point and return the corresponding value from specified column."""
    # Find the geometry that is closest
    nearest = df2[geom2_col] == nearest_points(row[geom1_col], geom_union)[1]
    # Get the corresponding value from df2 (matching is based on the geometry)
    value = df2[nearest][src_column].get_values()[0]
    return value

In [45]:
orel_stops = gpd.read_file(r'C:\Users\poletaev_im\nearest_stop_id_from_kml\Orel_routes\остановки.shp')
stop_sequence = pd.read_csv(r'C:\Users\poletaev_im\nearest_stop_id_from_kml\Orel_routes\stop_sequence.csv',sep=';', encoding='cp1251')
stop_sequence = stop_sequence.merge(orel_stops, how = 'left', on = 'STOP_ID')
stop_sequence = gpd.GeoDataFrame(stop_sequence, geometry=stop_sequence['geometry'])
stop_sequence

,TRIP_ID,STOP_ID,STOP_SEQUENCE,STOP_NAME,X,Y,geometry
0,2,80,1,Узловая улица,36.091041,52.931916,POINT (36.091041 52.931916)
1,2,83,2,СУ-5,36.083161,52.930165,POINT (36.083161 52.930165)
2,2,85,3,По требованию,36.079562,52.929203,POINT (36.079562 52.929203)
3,2,87,4,Завод Монолит,36.076858,52.928455,POINT (36.076858 52.928455)
4,2,211,5,Яблочная улица,36.072944,52.932954,POINT (36.072944 52.932954)
5,2,213,6,Лужковская улица,36.071277,52.934901,POINT (36.071277 52.934901)
6,2,194,7,Лесоторговая база,36.070673,52.940952,POINT (36.070673 52.940952)
7,2,191,8,ВОГ,36.074094,52.943499,POINT (36.074094 52.943499)
8,2,188,9,Скульптурная улица,36.075943,52.946030,POINT (36.075943 52.94603)
9,2,186,10,Городская улица,36.076884,52.949000,POINT (36.076884 52.949)


In [13]:
routes = gpd.read_file(r'C:\Users\poletaev_im\nearest_stop_id_from_kml\Orel_routes\Маршруты все + поля.shp')
routes = routes.reset_index().rename(columns = {'index':'route_number'})
routes

,route_number,TRIP_ID,ROUTE_NAME,ROUTE_LONG,TYPE,DIRECTION,geometry
0,0,2,1,Станция Лужки — ул. Часовая,автобус,0,(LINESTRING (36.08877959987726 52.930957199843...
1,1,3,2,Ул. Планерная - ул. Гайдара,автобус,0,(LINESTRING (36.09002369980318 52.993342700071...
2,2,4,3,Ливенская АЗС — Госуниверситет-УНПК,автобус,0,(LINESTRING (36.10984249969361 52.952633900252...
3,3,5,4,Ул. Алроса - «Госуниверситет — УНПК,автобус,0,(LINESTRING (36.03542700026664 52.991378200100...
4,4,6,5,Мегакомплекс ГРИНН - поселок Булгаково,автобус,0,(LINESTRING (36.01418770031594 52.921398200000...
5,5,7,6,Ул. Алроса - ул. Силикатная,автобус,0,(LINESTRING (36.10526989988764 53.009722400250...
6,6,8,8,Мегакомплекс ГРИНН — Наугорское шоссе,автобус,0,(LINESTRING (36.01418770031594 52.921398200000...
7,7,9,9,Микрорайон Зареченский -санаторий «Лесной»,автобус,0,(LINESTRING (36.14249009980944 53.021939899925...
8,8,10,10,ОАО Северсталь-метиз» - Наугорское шоссе,автобус,0,(LINESTRING (36.1762087002271 53.0454921000932...
9,9,11,11,Спецавтобаза – Веселая Слобода,автобус,0,(LINESTRING (36.12633689971118 52.970711799869...


In [5]:
start = gpd.GeoSeries(routes['geometry'])
start

0     (LINESTRING (36.08877959987726 52.930957199843...
1     (LINESTRING (36.09002369980318 52.993342700071...
2     (LINESTRING (36.10984249969361 52.952633900252...
3     (LINESTRING (36.03542700026664 52.991378200100...
4     (LINESTRING (36.01418770031594 52.921398200000...
5     (LINESTRING (36.10526989988764 53.009722400250...
6     (LINESTRING (36.01418770031594 52.921398200000...
7     (LINESTRING (36.14249009980944 53.021939899925...
8     (LINESTRING (36.1762087002271 53.0454921000932...
9     (LINESTRING (36.12633689971118 52.970711799869...
10    (LINESTRING (36.01016239976036 52.925314800261...
11    (LINESTRING (36.01418770031594 52.921398200000...
12    (LINESTRING (36.01537929964513 52.928101900104...
13    (LINESTRING (36.03542700026664 52.991378200100...
14    (LINESTRING (36.01537929964513 52.928101900104...
15    (LINESTRING (36.01537929964513 52.928101900104...
16    (LINESTRING (36.02496780005764 52.951464599868...
17    (LINESTRING (36.04721814622021 52.92944204

In [33]:
all_lines = gpd.GeoDataFrame()

for y in tqdm(range(len(start))):
    one_route = start.iloc[y]
    map_coords = mapping(one_route)
    type_line = map_coords['type']
    
    if type_line == 'LineString':
        
        list_coords = list(map_coords['coordinates'])
        df_of_coords = pd.DataFrame(list_coords)
        df_of_coords = pd.DataFrame(list_coords, columns = ['lon','lat'])
        df_of_coords = df_of_coords.reset_index().rename(columns = {'index':'nod_sequence'})
        df_of_coords['line_id'] = 0
        df_of_coords['route_number'] = y
        all_lines=all_lines.append(df_of_coords)
    else:
            
        list_coords = list(map_coords['coordinates'])
        df_of_coords = pd.DataFrame(list_coords)
        df_of_coords = df_of_coords.stack().reset_index()
        df_of_coords.columns = ['line_id','nod_sequence','geometry']
        df_of_coords['str_geo'] = df_of_coords['geometry'].astype(str)
        df_of_coords['lon'] = df_of_coords['str_geo'].str.split(',',n=1, expand=True)[0]
        df_of_coords['lon'] = df_of_coords.lon.str.replace("(", "")
        df_of_coords['lon'] = df_of_coords['lon'].astype(float)
        df_of_coords['lat'] = df_of_coords['str_geo'].str.split(',',n=1, expand=True)[1]
        df_of_coords['lat'] = df_of_coords.lat.str.replace(")", "")
        df_of_coords['lat'] = df_of_coords['lat'].astype(float)
        df_of_coords['route_number'] = y
        all_lines=all_lines.append(df_of_coords)
all_lines = all_lines.merge(routes[['route_number','TRIP_ID']], how = 'left', on = 'route_number')
all_lines = all_lines.drop(['str_geo','geometry'], axis=1)
geometry = [Point(xy) for xy in zip(all_lines.lon, all_lines.lat)]
all_lines = gpd.GeoDataFrame(all_lines, geometry=geometry)
all_lines
    

100%|██████████████████████████████████████████| 58/58 [00:00<00:00, 93.55it/s]


,line_id,nod_sequence,lon,lat,route_number,TRIP_ID,geometry
0,0,0,36.088780,52.930957,0,2,POINT (36.08877959987726 52.9309571998431)
1,0,1,36.090680,52.931770,0,2,POINT (36.09068000042949 52.93176999976377)
2,0,2,36.092130,52.932160,0,2,POINT (36.09213000003707 52.93215999974973)
3,0,3,36.092970,52.932380,0,2,POINT (36.0929699996919 52.93237999990114)
4,0,4,36.093940,52.932640,0,2,POINT (36.09393999988215 52.93263999987035)
5,0,5,36.094190,52.932704,0,2,POINT (36.09418973972641 52.93270388706965)
6,1,0,36.088780,52.930957,0,2,POINT (36.08877959987726 52.9309571998431)
7,1,1,36.088300,52.930790,0,2,POINT (36.08829999975872 52.93078999988472)
8,1,2,36.087720,52.930640,0,2,POINT (36.08772000027542 52.93063999993655)
9,1,3,36.083460,52.930210,0,2,POINT (36.08346000016305 52.93020999991472)


In [46]:
TRIP_ID = 2
one_seq = stop_sequence.query('TRIP_ID == @TRIP_ID').reset_index(drop=True)
one_line = all_lines.query('TRIP_ID == @TRIP_ID').reset_index(drop=True)
one_line = one_line.reset_index().rename(columns = {'index':'point_sequence'})
unary_union = all_lines.unary_union
kml_file = one_seq
kml_file['nearest_id'] = kml_file.apply(nearest, geom_union=unary_union, df1=one_seq, df2=one_line, geom1_col='geometry', src_column='point_sequence', axis=1)
one_line = one_line.merge(kml_file[['nearest_id','STOP_ID']], how = 'left', left_on = 'point_sequence', right_on = 'nearest_id')
kml_file

,TRIP_ID,STOP_ID,STOP_SEQUENCE,STOP_NAME,X,Y,geometry,nearest_id
0,2,80,1,Узловая улица,36.091041,52.931916,POINT (36.091041 52.931916),1
1,2,83,2,СУ-5,36.083161,52.930165,POINT (36.083161 52.930165),9
2,2,85,3,По требованию,36.079562,52.929203,POINT (36.079562 52.929203),10
3,2,87,4,Завод Монолит,36.076858,52.928455,POINT (36.076858 52.928455),11
4,2,211,5,Яблочная улица,36.072944,52.932954,POINT (36.072944 52.932954),24
5,2,213,6,Лужковская улица,36.071277,52.934901,POINT (36.071277 52.934901),32
6,2,194,7,Лесоторговая база,36.070673,52.940952,POINT (36.070673 52.940952),51
7,2,191,8,ВОГ,36.074094,52.943499,POINT (36.074094 52.943499),63
8,2,188,9,Скульптурная улица,36.075943,52.946030,POINT (36.075943 52.94603),57
9,2,186,10,Городская улица,36.076884,52.949000,POINT (36.076884 52.949),68


In [47]:
kml_file.to_file('kml_file.shp')